# Her skal vi undersøke dataen hentet og bruke Pandas/SQL for å forstå strukturen

### Her begynner vi med å forstå dataen fra frost bedre, og gjøre det om til en pandas dataframe, slik at det blir lett å jobbe med videre. 

I tillegg skal vi se på hvordan vi kan bruke pandasql (sqldf) for å gjøre querys på dataen og filtrere gjennom lettvint. Vi begynner med å hente file_path

In [21]:
import pandas as pd
from pandasql import sqldf
import os

# Henter file path

notebook_directory = os.getcwd()
root = os.path.abspath(os.path.join(notebook_directory, ".."))
file_path = os.path.join(root, "data", "Frost_Observations.json")



Dataen vi hentet er laget med mange dictionaries, som igjen inneholder flere dictionaries. For å kun få ut relevant data kan vi filtrere gjennom python for enkelte "entries" i forskjellige dictionaries. Deretter kan vi gjøre det om til en dataframe ved bruk av pandas og se hvordan dataen ser ut

In [22]:
import json

with open(file_path,"r") as fil: #leser filen og lagrer den som "data"
    data=json.load(fil)

# filtrerer dataen for bare det som er relevant til å analysere
brukbar_data = [
    {
        "elementId": obs["elementId"],
        "sourceId": entry["sourceId"],
        "referenceTime": entry["referenceTime"],
        "value": obs["value"],
        "unit": obs["unit"]
    }
    for entry in data["data"]
    for obs in entry["observations"] 
]

df=pd.DataFrame(brukbar_data) #Dataframe

pd.set_option('display.max_columns', None) #justerer output display
pd.set_option('display.width', 1000)
print(df.head())

                       elementId   sourceId             referenceTime  value     unit
0    mean(relative_humidity P1D)  SN18700:0  2020-04-01T00:00:00.000Z   56.0  percent
1  sum(precipitation_amount P1D)  SN18700:0  2020-04-01T00:00:00.000Z    0.0       mm
2  sum(precipitation_amount P1D)  SN18700:0  2020-04-01T00:00:00.000Z    0.0       mm
3           mean(wind_speed P1D)  SN18700:0  2020-04-01T00:00:00.000Z    2.6      m/s
4      mean(air_temperature P1D)  SN18700:0  2020-04-01T00:00:00.000Z    6.4     degC


Da har vi fått lagd en dataframe med pandas som viser relevant data. Det ser litt uryddig ut og det passer bedre å ha hver observasjon som kolonner og så verdiene nedover, så vi kan bruke pd.pivot til til å gjøre det mer ryddig. 

Men det at vi har 2 verdier for mean(air_temperature P1D) og for sum(precipitation_amount P1D) gjør at dette ikke fungerer. Foreløpig kan vi kombinere det ved å ta gjennomsnittsverdien av de 2 målte, slik at vi kan visualisere dataframen bedre. Vi kan bruke "groupby" funksjonen for å samle de 2 identiske elementene og slå sammen verdiene med gjennomsnittet. 

In [17]:
rader=[]

# henter relevant data ved å iterere gjennom i forskjellige dictionaries
for entry in data["data"]:
    referenceTime = entry["referenceTime"]
    for obs in entry["observations"]:
         rader.append({
            "referenceTime": referenceTime,
            "elementId": obs["elementId"],
            "value": obs["value"],
        })
         
df=pd.DataFrame(rader) #dataframe

# slår sammen der det er 2 av mean(air_temperature P1D) og tar gjennomsnittet, 
# gjør pivot av dataframe, slik at kolonnene blir type observasjon

df_gruppering = df.groupby(["referenceTime", "elementId"], as_index=False)["value"].mean()
df2=df_gruppering.pivot(index="referenceTime",columns="elementId",values="value")
df2=df2.reset_index()
print(df2.head())



elementId             referenceTime  mean(air_temperature P1D)  mean(relative_humidity P1D)  mean(wind_speed P1D)  sum(precipitation_amount P1D)
0          2020-04-01T00:00:00.000Z                       6.15                         56.0                   2.6                           0.00
1          2020-04-02T00:00:00.000Z                       4.75                         63.0                   4.7                           0.35
2          2020-04-03T00:00:00.000Z                       3.60                         36.0                   4.3                           0.35
3          2020-04-04T00:00:00.000Z                       2.90                         49.0                   3.4                           0.00
4          2020-04-05T00:00:00.000Z                       4.65                         90.0                   1.9                           1.10


In [18]:
# lagrer df2 som json fil for videre bruk
df2.to_json('../data/df2_data.json',indent=4)

Her ser vi dataen på en mer forståelig måte. Det er verdt å merke at "boolean_clear_sky_weather(cloud_area_fraction P1D)" er boolean verdier, så i datahåndteringen kan vi ta hensyn til det og omgjøre det fra float. 

## Bruker PANDAS SQL (sqldf) til å hente data fra DataFrame

Nå som vi har en fungerende DataFrame, kan vi hente diverse type data og spesifikk data ved bruk av Pandas SQL/sqldf: 

1) Gjennomsnittlig temperatur for Juli i årene tilgjengelig
2) Alle dager der 24 timers gjennomsnittstemperaturen er over 20 grader og der det ikke var skyete basert på "boolean_clear_sky_weather..."

In [19]:
pysqldf = lambda q: sqldf(q, globals())

query = """

    SELECT strftime('%Y',referenceTime) AS År,
    AVG("mean(air_temperature P1D)") AS avg_temperatur_juli
    FROM df2
    WHERE strftime('%m',referenceTime) = '07'
    GROUP BY År
"""
resultat=pysqldf(query)
print(resultat)

     År  avg_temperatur_juli
0  2020            14.993548
1  2021            19.930645
2  2022            17.524194
3  2023            16.337097
4  2024            16.812903


In [20]:
query2="""
    SELECT * FROM df2
    WHERE "mean(air_temperature P1D)">=20.0
    ORDER BY "mean(air_temperature P1D)" DESC;
"""
resultat=pysqldf(query2)
print(resultat)

               referenceTime  mean(air_temperature P1D)  mean(relative_humidity P1D)  mean(wind_speed P1D)  sum(precipitation_amount P1D)
0   2023-06-17T00:00:00.000Z                      24.60                         36.0                   1.9                           0.00
1   2020-06-19T00:00:00.000Z                      24.15                         49.0                   2.7                           0.00
2   2023-06-16T00:00:00.000Z                      24.00                         43.0                   2.2                           0.00
3   2023-06-15T00:00:00.000Z                      23.55                         44.0                   1.7                           0.00
4   2020-06-27T00:00:00.000Z                      23.50                         59.0                   1.8                           0.05
5   2020-06-20T00:00:00.000Z                      23.35                         63.0                   3.2                           0.25
6   2020-06-16T00:00:00.000Z      